In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.calibration.calibration import Calibration
from plaster.run.sigproc_v2.sigproc_v2_params import SigprocV2Params
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/app/jobs_folder/sigproc_v2_compare/")
run = job.calib_val0

In [ ]:
subject_id = run.ims_import.tsv_data.scope_name.lower()
calib_path = f"/app/{subject_id}.calib"
print(f"Subject Id is: {subject_id}\ncalib_path is: {calib_path}")

In [ ]:
# @REMOVE-FROM-TEMPLATE
calib_path = f"/app/val0.calib"

In [ ]:
channel_i = 0
divs = 5

In [ ]:
for ch_i in range(run.ims_import.n_channels):
    qdf = run.ims_import.qualities()
    quality = qdf[qdf.channel_i == ch_i].sort_values(["quality"])
    z.hist(quality.quality, _size_x=800, _size_y=150, f_title=f"Quality distribution channel {ch_i}")
    
    row_iz = utils.ispace(0, len(qdf), 3)
    ims = np.array([
        run.ims_import.ims[row.field_i, row.channel_i, row.cycle_i]    
        for row in qdf.iloc[row_iz].itertuples()
    ])
    
    with z(_cols=3, _cspan=np.percentile(ims, (30, 99))):
        names = ("worst", "median", "best")
        for im, name in zip(ims, names):
            z.im(im, f_title=f"Channel: {ch_i} {name}")

In [ ]:
# Visualize the best quality fields to sanity check that the masking is good
qdf = run.ims_import.qualities()
quality = (
    qdf[qdf.channel_i == channel_i]
    .sort_values(["field_i", "cycle_i"])["quality"]
    .values
    .reshape((run.ims_import.n_fields, run.ims_import.n_cycles))
)
best_field_iz = np.argsort(np.sum(quality, axis=1))[::-1][0:6].tolist()
worst_field_iz = np.argsort(np.sum(quality, axis=1))[::][0:6].tolist()

cy_ims = run.ims_import.ims[best_field_iz[0], channel_i]
masked_cy_ims = imops.stack_map(cy_ims, worker.mask_anomalies_im)
aln_offsets, aln_scores = worker.align(masked_cy_ims)
chcy_ims = np.array([masked_cy_ims])
chcy_ims = worker.composite_with_alignment_offsets_chcy_ims(chcy_ims, aln_offsets)

divs = 5
mask_radius = 10
with z(_cols=4, _size=250):
    reg_bg_mean = []
    for cy_i in range(run.ims_import.n_cycles):
        stats, fg_im, bg_im = worker.regional_bg_fg_stats(
            worker.mask_anomalies_im(run.ims_import.ims[best_field_iz[0], channel_i, cy_i]),
            kernel_width=1.5,
            divs=divs,
            mask_radius=mask_radius,
            return_ims=True
        )
        reg_bg_mean += [stats[:, :, 0]]
        z.im(fg_im)
        z.im(bg_im)
    reg_bg_mean = np.array(reg_bg_mean)
z.cols(reg_bg_mean[:, divs//2, divs//2], f_x_axis_label="cycle", f_y_axis_label="mean center background")

In [ ]:
calib = worker.calibrate(run.ims_import, progress=lambda i, j: print(i, j))
calib.set_subject_id(subject_id)

In [ ]:
# chcy_regional_psfs.shape
# n_channels = chcy_regional_psfs.shape[0]
# for ch_i in range(n_channels):
#     ims = np.mean(chcy_regional_psfs[ch_i], axis=0)
#     calib.add({f"regional_psf.instrument_channel[{ch_i}].{subject_id}": ims.tolist()})

In [ ]:
_ = calib.save(calib_path)